In [1]:
##                DEPENDENCIES AND PACKAGES TO BE IMPORTED

## uncomment and run the below few lines, only the first time you are running the notebook
'''!pip install wordcloud
!pip install fileupload
!pip install ipywidgets
!jupyter nbextension install --py --user fileupload
!jupyter nbextension enable --py fileupload'''


import wordcloud
import re
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display
import fileupload
import nltk

## uncomment and run the line below, only the first time you are running the notebook
#nltk.download()

from nltk.corpus import stopwords
from nltk import word_tokenize
import itertools
import seaborn as sns
from nltk.stem import WordNetLemmatizer

#nltk.download('wordnet')
from nltk.corpus import wordnet as wn

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm
import spacy
from spacy import displacy
NER = spacy.load("en_core_web_sm")
NER.max_length = 10000000

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:
      Successfully uninstalled typing-extensions-3.7.4.2
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [18]:
##          GLOBAL VARIABLES. CHANGE THEM, AS PER YOUR ENVIRONMENT.

path = 'C:/Users/SOHAM BAIJAL/Documents/NLP_Project/books/'
Books = ['','Crime&Punishment.txt', 'war&peace.txt' , 'Machiavelli_Volume_I.txt']
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")

FIRST PART : **NOUNS & VERBS**

In [4]:
def tokenize_and_lowercase(path, book):
    ## open the text document T
    file_ptr = open( str(path + book),'r',encoding='utf-8')  
    
    ## reading the text document T
    T = file_ptr.read()     
    
    
    ## tokenizing (converting the words and characters in the form of list) the text document 
    ## to remove /n /r and chapter number and names
    T_tokens = word_tokenize(T)
    
    ## gives the number of tokens in the list 'tokens'
    print("Number of tokens in",book,"are = ",len(T_tokens))  
    
    ## lowercasing tokens
    T_tokens = [i.lower() for i in T_tokens]
    
    
    ## returning tokens
    return T_tokens

In [5]:
def remove_punctuation_marks(book,tokens):
    useful = []
    for i in tokens:
        if re.search("[a-zA-Z]+", i) != None:
            useful.append(i)
    print("Number of tokens, after removing punctuation marks, in",book,"are = ",len(useful))
    return useful

In [6]:
def remove_stop_words(book,tokens):
    useful = []
    for word in tokens:
        if word in stop_words:
            pass
        else:
            useful.append(word)
    print("Number of tokens, after removing stop words, in",book,"are = ",len(useful))
    return useful

In [7]:
def lemmatize(book,tokens):
    ## lemmatizing tokens
    tokens=[lemmatizer.lemmatize(i) for i in tokens]
    print("lemmatized tokens of",book)
    return tokens

In [8]:
def preprocess_books_without_lemmatization():
    Book_tokens = [""]
    for i in range(1,4):
        book = Books[i]
        
        print("_"*7,"Preprocessing",book,"_"*7)
        Book_tokens.append(tokenize_and_lowercase(path, book ))
        
        Book_tokens[i]   =   remove_punctuation_marks(book, Book_tokens[i] )
        
        Book_tokens[i]   =   remove_stop_words(book,Book_tokens[i] )
        
        
        print()
        
    return Book_tokens

In [9]:
def preprocess_books():
    Book_tokens = [""]
    for i in range(1,4):
        book = Books[i]
        
        print("_"*7,"Preprocessing",book,"_"*7)
        Book_tokens.append(tokenize_and_lowercase(path, book ))
        
        Book_tokens[i]   =   remove_punctuation_marks(book, Book_tokens[i] )
        
        Book_tokens[i]   =   remove_stop_words(book,Book_tokens[i] )
        
        Book_tokens[i]   =   lemmatize(book, Book_tokens[i] )
        
        print()
        
    return Book_tokens

In [10]:
## a generator, used to POS Tag Book Tokens
def POS_Tagger(tokens):
    for word in tokens:
        tag_result = nltk.pos_tag(word.split())
        yield tag_result

In [11]:
def print_progress_bar(i,number_of_book_tokens):
    
    if(i==0):
        print("|",end="")
        
    elif(i % 10000 == 0):
        if(i!=10000):
            print("\b"*(len(str(((i-10000)*100)//number_of_book_tokens))+1),end="")
        print("#",end="")
        percentage = (i*100)//number_of_book_tokens
        print(percentage,"\b%",end="")
        
    elif(i == number_of_book_tokens - 1):
        print("#","\b|")

In [12]:
def tag_books(prepped_books):
    tagged_books_temp = [""]
    
    for count in range(1,4):
        
        number_of_book_tokens = len(prepped_books[count])
        tagger_generator_object = POS_Tagger(prepped_books[count])
        tagged_book = []
        
        print("\n___________ Tagging",Books[count],"___________")   
        print("Number of words to tag =",number_of_book_tokens)
        
        for i in range(number_of_book_tokens):
            tagged_book.append(next(tagger_generator_object)[0])
            
            if( i % 10000 == 0 or i == number_of_book_tokens - 1):
                print_progress_bar(i,number_of_book_tokens)
            
        tagged_books_temp.append(tagged_book)
        print("\n___________ Tagged",Books[count],"___________")
        
    print("\n___________ Tagged all books ___________ ")
    return tagged_books_temp

In [13]:
def nouns_and_verbs(tagged_book):
    nouns_verbs = {'nouns':[],'verbs':[]}
    
    for tagged_word in tagged_book:
        
        if(tagged_word[1] in ['NOUN','NN','NNS','NNP','NNPS']):
            nouns_verbs['nouns'].append(tagged_word[0])
            
        elif(tagged_word[1] in ['VB','VBD','VBG','VBN','VBP','VBZ','VERB']):
            nouns_verbs['verbs'].append(tagged_word[0])
            
    return nouns_verbs

In [14]:
def calculate_nouns_and_verbs(tagged_books):
    nouns_verbs = [""]
    
    for count in range(1,4):
        
        print("Creating Nouns and Verbs list for",Books[count])
        
        nouns_verbs.append(nouns_and_verbs(tagged_books[count]))
        
        print("Created Nouns and Verbs list for",Books[count])
        
    return nouns_verbs

In [15]:
def freq_calc(Books):
    word_count = ["",{},{},{}]
    for count in range(1,4):
        word_set = Books[count]
        for word in word_set:
            word_count[count][word] = word_count[count].get(word,0)+1
    return word_count

In [16]:
def term_freq(books, word_count):
    for count in range(1,4):
        length = len(books[count])
        for word in word_count[count].keys():
            word_count[count][word] = word_count[count][word] / length
    return word_count

In [19]:
prepped_books = preprocess_books()

_______ Preprocessing Crime&Punishment.txt _______
Number of tokens in Crime&Punishment.txt are =  252650
Number of tokens, after removing punctuation marks, in Crime&Punishment.txt are =  206988
Number of tokens, after removing stop words, in Crime&Punishment.txt are =  91436
lemmatized tokens of Crime&Punishment.txt

_______ Preprocessing war&peace.txt _______
Number of tokens in war&peace.txt are =  669616
Number of tokens, after removing punctuation marks, in war&peace.txt are =  568434
Number of tokens, after removing stop words, in war&peace.txt are =  270611
lemmatized tokens of war&peace.txt

_______ Preprocessing Machiavelli_Volume_I.txt _______
Number of tokens in Machiavelli_Volume_I.txt are =  157105
Number of tokens, after removing punctuation marks, in Machiavelli_Volume_I.txt are =  134052
Number of tokens, after removing stop words, in Machiavelli_Volume_I.txt are =  66031
lemmatized tokens of Machiavelli_Volume_I.txt



In [20]:
tagged_books = tag_books(prepped_books)


___________ Tagging Crime&Punishment.txt ___________
Number of words to tag = 91436
|#10#21#32#43#54#65#76#87#98%#|

___________ Tagged Crime&Punishment.txt ___________

___________ Tagging war&peace.txt ___________
Number of words to tag = 270611
|###11#14#18#22#25#29#33#36#40#44#48#51#55#59#62#66#70#73#77#81#84#88#92#96#99%#|

___________ Tagged war&peace.txt ___________

___________ Tagging Machiavelli_Volume_I.txt ___________
Number of words to tag = 66031
|#15#30#45#60#75#90%#|

___________ Tagged Machiavelli_Volume_I.txt ___________

___________ Tagged all books ___________ 


In [21]:
nouns_and_verbs_list = calculate_nouns_and_verbs(tagged_books)

Creating Nouns and Verbs list for Crime&Punishment.txt
Created Nouns and Verbs list for Crime&Punishment.txt
Creating Nouns and Verbs list for war&peace.txt
Created Nouns and Verbs list for war&peace.txt
Creating Nouns and Verbs list for Machiavelli_Volume_I.txt
Created Nouns and Verbs list for Machiavelli_Volume_I.txt


In [22]:
print(nouns_and_verbs_list[1]['nouns'])

['july', 'man', 'garret', 's.', 'place', 'hesitation', 'towards', 'k.', 'bridge', 'meeting', 'landlady', 'staircase', 'garret', 'roof', 'house', 'cupboard', 'room', 'landlady', 'garret', 'dinner', 'attendance', 'floor', 'time', 'pas', 'kitchen', 'door', 'stood', 'time', 'man', 'sick', 'scowl', 'feel', 'debt', 'landlady', 'afraid', 'meeting', 'abject', 'time', 'past', 'condition', 'hypochondria', 'become', 'absorbed', 'fellow', 'meeting', 'landlady', 'anyone', 'poverty', 'anxiety', 'position', 'weigh', 'matter', 'importance', 'desire', 'nothing', 'landlady', 'terror', 'stair', 'listen', 'irrelevant', 'gossip', 'demand', 'payment', 'threat', 'complaint', 'rack', 'brain', 'excuse', 'prevaricate', 'lie', 'creep', 'stair', 'cat', 'slip', 'street', 'fear', 'want', 'attempt', 'thing', '_like', 'that_', 'trifle', 'thought', 'odd', 'smile', 'hm', 'yes', 'man', 'hand', 'slip', 'cowardice', 'axiom', 'men', 'afraid', 'step', 'word', 'fear', 'chatter', 'nothing', 'chatter', 'nothing', 'chatter', 'm

In [23]:
def give_categories_freq_distr(word):
    parent = {}
    for ss in wn.synsets(word):
        for hyper in ss.hypernyms():
            for items in hyper.lemma_names():
                parent[items] = parent.get(items,0)+1
    return parent

In [24]:
def category_determiner(words_list):
    set_of_words = set(words_list['nouns']+words_list['verbs'])
    word_category_frequency = {}
    for word in set_of_words:
        dict_of_categories = give_categories_freq_distr(word)
        for key in dict_of_categories.keys():
            word_category_frequency[key] = word_category_frequency.get(key,0)+ dict_of_categories[key]
    return word_category_frequency

In [25]:
word_cat_freq1 =  category_determiner(nouns_and_verbs_list[1])
word_cat_freq2 =  category_determiner(nouns_and_verbs_list[2])
word_cat_freq3 =  category_determiner(nouns_and_verbs_list[3])

In [26]:
print(word_cat_freq2)

{'flow': 62, 'flux': 18, 'move': 1439, 'change': 1050, 'hypothesis': 4, 'possibility': 6, 'theory': 5, 'opinion': 17, 'view': 78, 'reasoning': 8, 'logical_thinking': 8, 'abstract_thought': 8, 'expect': 43, 'anticipate': 45, 'coldness': 4, 'cold': 3, 'low_temperature': 3, 'frigidity': 4, 'frigidness': 4, 'composure': 5, 'calm': 19, 'calmness': 8, 'equanimity': 5, 'alter': 595, 'modify': 578, 'change_state': 164, 'turn': 290, 'current': 6, 'stream': 20, 'run': 153, 'feed': 79, 'course': 47, 'bridge': 1, 'sale': 5, 'sell': 32, 'control': 163, 'command': 90, 'direct': 117, 'orient': 4, 'orientate': 3, 'be': 674, 'trust': 29, 'swear': 45, 'rely': 23, 'bank': 26, 'see': 189, 'consider': 104, 'reckon': 93, 'regard': 57, 'add': 33, 'permit': 70, 'allow': 69, 'let': 67, 'countenance': 68, 'somesthesia': 6, 'somaesthesia': 6, 'somatesthesia': 6, 'somatic_sensation': 6, 'itch': 4, 'press': 55, 'displace': 349, 'compel': 31, 'oblige': 34, 'obligate': 34, 'overcharge': 9, 'soak': 14, 'surcharge': 9

**HISTOGRAM DRAWING**

In [27]:
result = word_cat_freq1
plt.bar(range(len(result.keys())), list(result.values()), align='center',width=0.3)
plt.xticks(range(len(result)), list(result.keys()),rotation='vertical')
plt.figure(figsize=(100,100))
plt.show()

KeyboardInterrupt: 

SECOND PART : **NER**

In [28]:
def N_E_R(path, book):
    NER_tags = {}
    file_ptr = open( str(path + book),'r',encoding='utf-8')  
    raw_text = file_ptr.read()
    text1= NER(raw_text)
    for word in text1.ents:
        NER_tags[word.label_] = NER_tags.get(word.label_,[])+[word.text]
    return NER_tags

In [29]:
def NER_on_books(Books):
    NER = [""]
    for count in range(1,4):
        print("NER tagging on",Books[count],"started")
        NER.append(N_E_R(path,Books[count]))
        print("NER tagging on",Books[count],"done")
    return NER

In [ ]:
N_E_R_dict = NER_on_books(Books)

NER tagging on Crime&Punishment.txt started
NER tagging on Crime&Punishment.txt done
NER tagging on war&peace.txt started
NER tagging on war&peace.txt done
NER tagging on Machiavelli_Volume_I.txt started


THIRD PART : **TF-IDF**

In [15]:
prepped_books = preprocess_books()
prepped_books_without_lemma  = preprocess_books_without_lemmatization()

_______ Preprocessing Crime&Punishment.txt _______
Number of tokens in Crime&Punishment.txt are =  252650
Number of tokens, after removing punctuation marks, in Crime&Punishment.txt are =  206988
Number of tokens, after removing stop words, in Crime&Punishment.txt are =  91436
lemmatized tokens of Crime&Punishment.txt

_______ Preprocessing war&peace.txt _______
Number of tokens in war&peace.txt are =  669616
Number of tokens, after removing punctuation marks, in war&peace.txt are =  568434
Number of tokens, after removing stop words, in war&peace.txt are =  270611
lemmatized tokens of war&peace.txt

_______ Preprocessing Machiavelli_Volume_I.txt _______
Number of tokens in Machiavelli_Volume_I.txt are =  157105
Number of tokens, after removing punctuation marks, in Machiavelli_Volume_I.txt are =  134052
Number of tokens, after removing stop words, in Machiavelli_Volume_I.txt are =  66031
lemmatized tokens of Machiavelli_Volume_I.txt

_______ Preprocessing Crime&Punishment.txt _______


In [17]:
word_count_without_lemmatization = freq_calc(prepped_books_without_lemma)
word_count_with_lemmatization = freq_calc(prepped_books)

In [23]:
print(word_count_without_lemmatization[1])

{'exceptionally': 4, 'hot': 13, 'evening': 56, 'early': 9, 'july': 1, 'young': 112, 'man': 470, 'came': 240, 'garret': 10, 'lodged': 5, 's.': 3, 'place': 112, 'walked': 102, 'slowly': 33, 'though': 440, 'hesitation': 10, 'towards': 40, 'k.': 1, 'bridge': 25, 'successfully': 2, 'avoided': 8, 'meeting': 29, 'landlady': 70, 'staircase': 32, 'roof': 1, 'high': 26, 'five-storied': 1, 'house': 108, 'like': 452, 'cupboard': 9, 'room': 276, 'provided': 6, 'dinners': 2, 'attendance': 1, 'lived': 20, 'floor': 36, 'every': 120, 'time': 376, 'went': 354, 'obliged': 8, 'pass': 41, 'kitchen': 21, 'door': 224, 'invariably': 2, 'stood': 175, 'open': 90, 'passed': 76, 'sick': 45, 'frightened': 54, 'feeling': 76, 'made': 229, 'scowl': 2, 'feel': 67, 'ashamed': 37, 'hopelessly': 10, 'debt': 12, 'afraid': 90, 'cowardly': 3, 'abject': 1, 'quite': 186, 'contrary': 24, 'past': 34, 'overstrained': 6, 'irritable': 15, 'condition': 21, 'verging': 1, 'hypochondria': 3, 'become': 41, 'completely': 39, 'absorbed':

**TERM FREQ with Lemmatization**

In [31]:
freq_count_w_lemma = term_freq(prepped_books, word_count_with_lemmatization)

**TERM FREQ without Lemmatization**

In [32]:
freq_count_without_lemma = term_freq(prepped_books_without_lemma, word_count_without_lemmatization)

**IDF CALCULATION**

In [39]:
import math

In [40]:
all_words = list(set(list(freq_count_w_lemma[1].keys()) + list(freq_count_w_lemma[2].keys()) + list(freq_count_w_lemma[3].keys())))

In [53]:
all_words_without_lemma = list(set(list(freq_count_without_lemma[1].keys()) + list(freq_count_without_lemma[2].keys()) + list(freq_count_without_lemma[3].keys())))

**IDF with Lemmatization**

In [43]:
idf_score = {}
for word in all_words:
    count = (1 if word in list(freq_count_w_lemma[1].keys()) else 0 ) + (1 if word in list(freq_count_w_lemma[3].keys()) else 0 ) + (1 if word in list(freq_count_w_lemma[2].keys()) else 0)
    df = 3.0 / count
    idf_score[word] = math.log(df,2)                                                                                                                       

**IDF without Lemmatization**

In [54]:
idf_score_without_lemma = {}
for word in all_words_without_lemma :
    count = (1 if word in list(freq_count_without_lemma[1].keys()) else 0 ) + (1 if word in list(freq_count_without_lemma[3].keys()) else 0 ) + (1 if word in list(freq_count_without_lemma[2].keys()) else 0)
    df = 3.0 / count
    idf_score_without_lemma[word] = math.log(df,2)  

**TF_IDF with Lemmatization**

In [49]:
tf_idf_with_lemma = ["",{},{},{}]
for count in range(1,4):
    for word in freq_count_w_lemma[count].keys():
        tf_idf_with_lemma[count][word] = freq_count_w_lemma[count][word] * idf_score[word]

**TF_IDF without Lemmatization**

In [57]:
tf_idf_without_lemma = ["",{},{},{}]
for count in range(1,4):
    for word in freq_count_without_lemma[count].keys():
        tf_idf_without_lemma[count][word] = freq_count_without_lemma[count][word] * idf_score_without_lemma[word]

**Cosine Similarity without Lemmatization**

In [68]:
vector1_mod = 0 
for word in tf_idf_without_lemma[1].keys():
    vector1_mod += math.pow(tf_idf_without_lemma[1][word],2)
vector1_mod = math.pow(vector1_mod, 0.5)

vector2_mod = 0 
for word in tf_idf_without_lemma[2].keys():
    vector2_mod += math.pow(tf_idf_without_lemma[2][word],2)
vector2_mod = math.pow(vector2_mod, 0.5)

vector3_mod = 0 
for word in tf_idf_without_lemma[3].keys():
    vector3_mod += math.pow(tf_idf_without_lemma[3][word],2)
vector3_mod = math.pow(vector3_mod, 0.5)

In [69]:
## 1 and 2
simi = 0
for word in tf_idf_without_lemma[1].keys():
    if(tf_idf_without_lemma[1][word] == 0):
        pass
    else:
        simi += tf_idf_without_lemma[1][word]*tf_idf_without_lemma[2].get(word,0.0)
simi = simi / (vector1_mod * vector2_mod)
print("cosine similarity of book 1 and 2 without lemmatization =",simi)
## 2 and 3
simi = 0
for word in tf_idf_without_lemma[2].keys():
    if(tf_idf_without_lemma[2][word] == 0):
        pass
    else:
        simi += tf_idf_without_lemma[2][word]*tf_idf_without_lemma[3].get(word,0.0)
simi = simi / (vector2_mod * vector3_mod)
print("cosine similarity of book 2 and 3 without lemmatization =",simi)
## 1 and 3
simi = 0
for word in tf_idf_without_lemma[1].keys():
    if(tf_idf_without_lemma[1][word] == 0):
        pass
    else:
        simi += tf_idf_without_lemma[1][word]*tf_idf_without_lemma[3].get(word,0.0)
simi = simi / (vector1_mod * vector3_mod)
print("cosine similarity of book 1 and 3 without lemmatization =",simi)

cosine similarity of book 1 and 2 without lemmatization = 0.02938159597942474
cosine similarity of book 2 and 3 without lemmatization = 0.0021617665129280484
cosine similarity of book 1 and 3 without lemmatization = 0.00022519040216826725


**Cosine Similarity with Lemmatization**

In [71]:
vector1_mod = 0 
for word in tf_idf_with_lemma[1].keys():
    vector1_mod += math.pow(tf_idf_with_lemma[1][word],2)
vector1_mod = math.pow(vector1_mod, 0.5)

vector2_mod = 0 
for word in tf_idf_with_lemma[2].keys():
    vector2_mod += math.pow(tf_idf_with_lemma[2][word],2)
vector2_mod = math.pow(vector2_mod, 0.5)

vector3_mod = 0 
for word in tf_idf_with_lemma[3].keys():
    vector3_mod += math.pow(tf_idf_with_lemma[3][word],2)
vector3_mod = math.pow(vector3_mod, 0.5)

In [72]:
## 1 and 2
simi = 0
for word in tf_idf_with_lemma[1].keys():
    if(tf_idf_with_lemma[1][word] == 0):
        pass
    else:
        simi += tf_idf_with_lemma[1][word]*tf_idf_with_lemma[2].get(word,0.0)
simi = simi / (vector1_mod * vector2_mod)
print("cosine similarity of book 1 and 2 with lemmatization =",simi)

## 2 and 3
simi = 0
for word in tf_idf_with_lemma[2].keys():
    if(tf_idf_with_lemma[2][word] == 0):
        pass
    else:
        simi += tf_idf_with_lemma[2][word]*tf_idf_with_lemma[3].get(word,0.0)
simi = simi / (vector2_mod * vector3_mod)
print("cosine similarity of book 2 and 3 with lemmatization =",simi)

## 1 and 3
simi = 0
for word in tf_idf_with_lemma[1].keys():
    if(tf_idf_with_lemma[1][word] == 0):
        pass
    else:
        simi += tf_idf_with_lemma[1][word]*tf_idf_with_lemma[3].get(word,0.0)
simi = simi / (vector1_mod * vector3_mod)
print("cosine similarity of book 1 and 3 with lemmatization =",simi)

cosine similarity of book 1 and 2 with lemmatization = 0.030176206298803382
cosine similarity of book 2 and 3 with lemmatization = 0.0026759154043093877
cosine similarity of book 1 and 3 with lemmatization = 0.00023992950732353255
